# 卷积神经网络_GPU set_memory_growth设置GPU内存增长

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
print(tf.__version__)
print(sys.version_info)
for module in np, pd ,sklearn, tf, keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [3]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus))
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(logical_gpus))


0
0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
        x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_valid_scaled = scaler.transform(
        x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_test_scaled = scaler.transform(
        x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)

In [ ]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    # prefetch 从数据中预先取出多少个给训练集作准备
    return dataset

batch_size = 120
epochs = 100
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)
        

In [ ]:
# kernel_size卷积窗口的长度
# pool_size池化窗口的长度
# 池化操作通常不重叠，不补0.pool_size=步长，用于减小图像尺寸，减少计算量
# filter输出的通道，即卷积核的数量
# padding='same'指填充padding使得输入输出大小相同，默认‘valid’指不填充
# strides=(1, 1)卷积时在每一维度上的步长
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                        input_shape =[28,28,1], activation='relu'),
    keras.layers.Conv2D(filters = 32,kernel_size=3,padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                         activation='relu'),
    keras.layers.Conv2D(filters = 64,kernel_size=3,padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                        activation='relu'),
    keras.layers.Conv2D(filters = 128,kernel_size=3,padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),#连接全连接层前要flatten展平
    keras.layers.Dense(128, activation= 'relu'),
    keras.layers.Dense(10,activation='softmax')
])    

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer = 'adam',
             metrics = ['accuracy'])


In [ ]:
model.layers#查看模型层数

In [ ]:
model.summary()

In [ ]:

history = model.fit(train_dataset, steps_per_epoch = x_test_scaled.shape[0]//batch_size,  epochs=10)


In [ ]:
history.history

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=[8,5])
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

plot_learning_curves(history)